In [1]:
import jupyros as jr
import rospy
import tf
import numpy as np
import ipywidgets as widgets
import math

from geometry_msgs.msg import PoseStamped
from tf.transformations import euler_from_quaternion
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

from robotcontroller import RobotController

rospy.init_node('jupyter_node')

In [2]:
jr.publish('/cmd_vel', Twist)

In [3]:
mapframe = 'odom'
transformlistener = tf.TransformListener()
cmdvelpub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

robotcontroller = RobotController(transformlistener, mapframe, cmdvelpub)
jr.subscribe('/odom', Odometry, robotcontroller.newodometry)

In [4]:
output = widgets.Output()

def slider_change(change):
    with output:
        newvalue = change['new']
        print(newvalue)    
        robotcontroller.appendbehavior(RotateToState(newvalue))
    return

slider = widgets.FloatSlider(
    value=1,
    min=-math.pi,
    max=math.pi,
    step=0.1,
    description='Target theta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
slider.observe(slider_change, names='value')

display(slider, output)

FloatSlider(value=1.0, continuous_update=False, description='Target theta:', max=3.141592653589793, min=-3.141…

Output()

In [5]:
from rotatetostate import RotateToState
from drivetoposition import DriveToPosition

def navgoal(message):

    if robotcontroller.latestodominmapframe:
        
        navgoalinframe = transformlistener.transformPose(mapframe, message)
        
        dx = navgoalinframe.pose.position.x - robotcontroller.latestodominmapframe.pose.position.x
        dy = navgoalinframe.pose.position.y - robotcontroller.latestodominmapframe.pose.position.y
        
        yaw = math.atan2(dy, dx)
        robotcontroller.appendbehavior(DriveToPosition(navgoalinframe))
        robotcontroller.appendbehavior(RotateToState(yaw))

    return

jr.subscribe('/move_base_simple/goal', PoseStamped, navgoal)

In [34]:
from nav_msgs.msg import OccupancyGrid
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=300, height=500)
display(canvas)

def newnavgrid(message):
    
    try:    
        with hold_canvas():  
            canvas.clear()
            canvas.layout.width = str(message.info.width) + 'px'
            canvas.layout.height = str(message.info.height) + 'px'
            canvas.width = message.info.width
            canvas.height = message.info.height
            
            if True:
                for y in range(message.info.height):
                    for x in range(message.info.width):
                        bottomx = x;
                        bottomy = message.info.height - 1 - y

                        value = message.data[y * message.info.width + x]

                        if value == 0:
                            # Free cell
                            canvas.fill_style = "lightgray"
                            canvas.stroke_style = "lightgray"

                            canvas.fill_rect(bottomx, bottomy - 1, 1, 1)
                        if value > 0:
                            # Occupied
                            canvas.fill_style = "black"
                            canvas.stroke_style = "black"

                            canvas.fill_rect(bottomx, bottomy - 1, 1, 1)

            if robotcontroller.latestodominmapframe:
                robotx = robotcontroller.latestodominmapframe.pose.position.x
                roboty = robotcontroller.latestodominmapframe.pose.position.y

                mapx = int((robotx - message.info.origin.position.x) / message.info.resolution)
                mapy = int((roboty - message.info.origin.position.y) / message.info.resolution)

                canvas.fill_style = "red"
                canvas.stroke_style = "red"

                bottomy = message.info.height - 1 - mapy
                canvas.fill_rect(mapx, bottomy + 1, 4, 4)

    except Exception as e:
        rospy.logerr('Error processing map data : %s', e)


jr.subscribe('/costmap_node/costmap/costmap', OccupancyGrid, newnavgrid)

Canvas(width=300)

Removing previous callback, only one redirection possible right now
